In [267]:
import requests
import json
import pandas as pd

In [268]:
pd.options.display.max_columns = None

### Step 1. Prepare the plumbing

In [269]:
resource = 'https://xxxxx.crm6.dynamics.com'
clientId = '00000000000000000000000000000000000'
tenantId = '11111111111111111111111111111111111'
grantType = 'client_credentials'
clientSecret = '22222222222222222222222222222222'
tokenendpoint = 'https://login.microsoftonline.com/11111111111111111111111111111111/oauth2/token'

In [152]:
base_url = 'https://xxxxxxx.crm6.dynamics.com/api/data/v9.2'

In [270]:
audits_url = '/bi_projects'

In [271]:
base_url + audits_url

'https://orgfc831d9e.crm6.dynamics.com/api/data/v9.2/bi_projects'

In [272]:
tokenpost = {
    'client_id':clientId,
    'tenant_id': tenantId,
    'resource':resource,
    'client_secret': clientSecret,
    'grant_type': grantType,
}

In [273]:
tokenres = requests.post(tokenendpoint, data=tokenpost)

In [274]:
accesstoken = ''

In [275]:
try:
    accesstoken = tokenres.json()['access_token']
except(KeyError):
    #handle any missing key errors
    print('Could not get access token')

In [276]:
request_header = {
    'Authorization': 'Bearer ' + accesstoken,
    'OData-MaxVersion': '4.0',
    'OData-Version': '4.0',
    'Accept': 'application/json',
    'Content-Type': 'application/json; charset=utf-8',
    'Prefer': 'odata.maxpagesize=500',
    'Prefer': 'odata.include-annotations=OData.Community.Display.V1.FormattedValue'
    }

In [277]:
def get(endpoint, query = '', df = None):
    data = requests.get(endpoint + query, headers=request_header).json()
    if df is None:
        df = pd.DataFrame()
    if data is not None:
        if 'value' in data:
            df = df.append(pd.DataFrame.from_records(data['value']))
        if '@odata.nextLink' in data:
            query = urlparse(data['@odata.nextLink']).query
        return df

### Step 2. Get the 'Projects' table

In [280]:
projects = get(endpoint=base_url + audits_url, query="")

In [282]:
projects.head()

,@odata.etag,statecode@OData.Community.Display.V1.FormattedValue,statecode,bi_projectareasingle@OData.Community.Display.V1.FormattedValue,bi_projectareasingle,bi_budget@OData.Community.Display.V1.FormattedValue,bi_budget,_bi_pm_name_value@OData.Community.Display.V1.FormattedValue,_bi_pm_name_value,bi_projectstatus@OData.Community.Display.V1.FormattedValue,bi_projectstatus,createdon@OData.Community.Display.V1.FormattedValue,createdon,bi_projectenddate@OData.Community.Display.V1.FormattedValue,bi_projectenddate,bi_projectid,_ownerid_value@OData.Community.Display.V1.FormattedValue,_ownerid_value,modifiedon@OData.Community.Display.V1.FormattedValue,modifiedon,versionnumber@OData.Community.Display.V1.FormattedValue,versionnumber,timezoneruleversionnumber@OData.Community.Display.V1.FormattedValue,timezoneruleversionnumber,_bi_projectsponsor_value@OData.Community.Display.V1.FormattedValue,_bi_projectsponsor_value,bi_totalvariation_state@OData.Community.Display.V1.FormattedValue,bi_totalvariation_state,statuscode@OData.Community.Display.V1.FormattedValue,statuscode,_modifiedby_value@OData.Community.Display.V1.FormattedValue,_modifiedby_value,bi_projectname,bi_totalvariation@OData.Community.Display.V1.FormattedValue,bi_totalvariation,bi_totalvariation_date@OData.Community.Display.V1.FormattedValue,bi_totalvariation_date,bi_expectedcompletiondate@OData.Community.Display.V1.FormattedValue,bi_expectedcompletiondate,bi_forecast@OData.Community.Display.V1.FormattedValue,bi_forecast,_createdby_value@OData.Community.Display.V1.FormattedValue,_createdby_value,bi_description,bi_businesscaseprovided@OData.Community.Display.V1.FormattedValue,bi_businesscaseprovided,_owningbusinessunit_value@OData.Community.Display.V1.FormattedValue,_owningbusinessunit_value,bi_governancecommittee@OData.Community.Display.V1.FormattedValue,bi_governancecommittee,_owninguser_value,bi_projectstartdate@OData.Community.Display.V1.FormattedValue,bi_projectstartdate,bi_resourcesinvolved@OData.Community.Display.V1.FormattedValue,bi_resourcesinvolved,bi_fundedbudget@OData.Community.Display.V1.FormattedValue,bi_fundedbudget,bi_budgetspent@OData.Community.Display.V1.FormattedValue,bi_budgetspent,_owningteam_value,importsequencenumber,utcconversiontimezonecode,bi_projectteam,bi_totalbudget,bi_projid,bi_boardapprovaldates,_createdonbehalfby_value,bi_closedreason,_modifiedonbehalfby_value,bi_projectarea,overriddencreatedon,bi_totalbudget@OData.Community.Display.V1.FormattedValue,_modifiedonbehalfby_value@OData.Community.Display.V1.FormattedValue,bi_projectarea@OData.Community.Display.V1.FormattedValue
0,"W/""4396606""",Active,0,Strategy,100000004.0,"111,111.00",111111.0,Aditya Ambale,9ab75eb9-3282-ec11-8d21-002248929dc9,Starting,100000000,1/6/2022 12:31 AM,2022-01-06T00:31:43Z,12/29/2021,2021-12-29,c9ca1505-886e-ec11-8943-000d3a6b5be3,Aditya Ambale Admin,64a73127-c5ff-eb11-94ef-0022481063de,3/21/2022 4:48 AM,2022-03-21T04:48:20Z,"4,396,606",4396606,0,0,Gavin Masters,806059bf-3282-ec11-8d21-002248929dc9,1,1,Active,1,Robin Du Admin,d6ec6dbc-be53-ec11-8f8e-00224811ce87,Outcome Measurement,0.00,0.0,4/4/2022 5:38 PM,2022-04-04T17:38:30Z,12/30/2022,2022-12-30,"4,444.00",4444.0,Aditya Ambale Admin,64a73127-c5ff-eb11-94ef-0022481063de,this is another test from canvas app alksdfj l...,Yes,100000000.0,orgfc831d9e,ed2ac57e-9cf7-eb11-94ee-000d3acc5224,PCG,100000001.0,64a73127-c5ff-eb11-94ef-0022481063de,6/5/2021,2021-06-05,Both,100000002.0,"2,222.00",2222.0,"3,333.00",3333.0,None,None,None,None,NaN,None,None,None,None,None,None,None,NaN,NaN,NaN
1,"W/""4396607""",Active,0,Capital Projects,100000003.0,"150,000.00",150000.0,Robin Du,cb2f9386-e86c-ec11-8943-000d3a6b587c,Managing,100000002,12/13/2021 1:28 AM,2021-12-13T01:28:03Z,12/22/2021,2021-12-22,390732e6-b35b-ec11-8f8f-000d3ad20434,Aditya Ambale Admin,64a73127-c5ff-eb11-94ef-0022481063de,3/31/2022 5:22 AM,2022-03-31T05:22:44Z,"4,396,607",4396607,0,0,Robin Du,cb2f9386-e86c-ec11-8943-000d3a6b587c,1,1,Active,1,Aditya Ambale Admin,64a73127-c5f

In [286]:
test = projects.loc[projects['bi_projectname'].str.contains('Capital')==True]

In [287]:
test

,@odata.etag,statecode@OData.Community.Display.V1.FormattedValue,statecode,bi_projectareasingle@OData.Community.Display.V1.FormattedValue,bi_projectareasingle,bi_budget@OData.Community.Display.V1.FormattedValue,bi_budget,_bi_pm_name_value@OData.Community.Display.V1.FormattedValue,_bi_pm_name_value,bi_projectstatus@OData.Community.Display.V1.FormattedValue,bi_projectstatus,createdon@OData.Community.Display.V1.FormattedValue,createdon,bi_projectenddate@OData.Community.Display.V1.FormattedValue,bi_projectenddate,bi_projectid,_ownerid_value@OData.Community.Display.V1.FormattedValue,_ownerid_value,modifiedon@OData.Community.Display.V1.FormattedValue,modifiedon,versionnumber@OData.Community.Display.V1.FormattedValue,versionnumber,timezoneruleversionnumber@OData.Community.Display.V1.FormattedValue,timezoneruleversionnumber,_bi_projectsponsor_value@OData.Community.Display.V1.FormattedValue,_bi_projectsponsor_value,bi_totalvariation_state@OData.Community.Display.V1.FormattedValue,bi_totalvariation_state,statuscode@OData.Community.Display.V1.FormattedValue,statuscode,_modifiedby_value@OData.Community.Display.V1.FormattedValue,_modifiedby_value,bi_projectname,bi_totalvariation@OData.Community.Display.V1.FormattedValue,bi_totalvariation,bi_totalvariation_date@OData.Community.Display.V1.FormattedValue,bi_totalvariation_date,bi_expectedcompletiondate@OData.Community.Display.V1.FormattedValue,bi_expectedcompletiondate,bi_forecast@OData.Community.Display.V1.FormattedValue,bi_forecast,_createdby_value@OData.Community.Display.V1.FormattedValue,_createdby_value,bi_description,bi_businesscaseprovided@OData.Community.Display.V1.FormattedValue,bi_businesscaseprovided,_owningbusinessunit_value@OData.Community.Display.V1.FormattedValue,_owningbusinessunit_value,bi_governancecommittee@OData.Community.Display.V1.FormattedValue,bi_governancecommittee,_owninguser_value,bi_projectstartdate@OData.Community.Display.V1.FormattedValue,bi_projectstartdate,bi_resourcesinvolved@OData.Community.Display.V1.FormattedValue,bi_resourcesinvolved,bi_fundedbudget@OData.Community.Display.V1.FormattedValue,bi_fundedbudget,bi_budgetspent@OData.Community.Display.V1.FormattedValue,bi_budgetspent,_owningteam_value,importsequencenumber,utcconversiontimezonecode,bi_projectteam,bi_totalbudget,bi_projid,bi_boardapprovaldates,_createdonbehalfby_value,bi_closedreason,_modifiedonbehalfby_value,bi_projectarea,overriddencreatedon,bi_totalbudget@OData.Community.Display.V1.FormattedValue,_modifiedonbehalfby_value@OData.Community.Display.V1.FormattedValue,bi_projectarea@OData.Community.Display.V1.FormattedValue
3,"W/""4395681""",Active,0,Infrastructure,100000000.0,"4,801,566.60",4801566.6,Robin Du,cb2f9386-e86c-ec11-8943-000d3a6b587c,Initiating,100000001,12/10/2021 12:10 AM,2021-12-10T00:10:07Z,1/23/2022,2022-01-23,1e187487-4d59-ec11-8f8f-00224811ae90,Robin Du,c669fb53-3277-ec11-8d21-0022489225eb,4/4/2022 6:03 AM,2022-04-04T06:03:56Z,"4,395,681",4395681,0,0,NaN,None,1,1,Active,1,Aditya Ambale Admin,64a73127-c5ff-eb11-94ef-0022481063de,Capital Building 1,"1,100.00",1100.0,4/4/2022 5:38 PM,2022-04-04T17:38:30Z,3/15/2022,2022-03-15,"5,201,000.00",5201000.0,Aditya Ambale Admin,64a73127-c5ff-eb11-94ef-0022481063de,None,Yes,100000000.0,orgfc831d9e,ed2ac57e-9cf7-eb11-94ee-000d3acc5224,PCG,100000001.0,c669fb53-3277-ec11-8d21-0022489225eb,7/4/2021,2021-07-04,NaN,NaN,NaN,NaN,"1,500,000.00",1500000.0,None,None,None,Infrastructure,4802666.6,None,None,None,None,None,None,None,"4,802,666.60",NaN,NaN


## Step 3. Excecute and Retrieve Calculate Rollup Field

In [179]:
sample = "https://crm.com/api/data/v8.0/CalculateRollupField(Target=@tid,FieldName=@field)?@tid={'@odata.id':'ccseq_contracts(3047FD4D-CCC3-E711-80E9-005056B33317)'}&@field='ccseq_totalamount'"

In [288]:
url = "https://orgfc831d9e.crm6.dynamics.com/api/data/v9.2/CalculateRollupField(Target=@tid,FieldName=@field)?@tid={'@odata.id':'bi_projects(1e187487-4d59-ec11-8f8f-00224811ae90)'}&@field='bi_totalvariation'"

In [289]:
result = requests.get(url, headers=request_header).json()

In [290]:
result

{'@odata.context': 'https://orgfc831d9e.crm6.dynamics.com/api/data/v9.2/$metadata#bi_projects/$entity',
 '@odata.type': '#Microsoft.Dynamics.CRM.bi_project',
 '@odata.etag': 'W/"4396771"',
 'bi_totalvariation@OData.Community.Display.V1.FormattedValue': '1,300.00',
 'bi_totalvariation': 1300.0,
 'bi_totalvariation_date@OData.Community.Display.V1.FormattedValue': '4/4/2022 11:33 PM',
 'bi_totalvariation_date': '2022-04-04T23:33:23Z',
 'bi_totalvariation_state@OData.Community.Display.V1.FormattedValue': '1',
 'bi_totalvariation_state': 1,
 'bi_projectid': '1e187487-4d59-ec11-8f8f-00224811ae90'}